In [ ]:
# change to the root directory of the project
import os
if os.getcwd().split("/")[-1] == "examples":
    os.chdir('..')
print(os.getcwd())

If we want to run this notebook on Google Colab, we first have to install `ScanDy` and download the required dataset from Google drive. The following code cell will prepare all of this for us.

In [ ]:
# install the ScanDy framework via pip
!pip install scandy

# download the VidCom_example dataset from google drive using gdown
!pip install gdown
# dataset is stored at https://drive.google.com/file/d/1oT9OJ2tRsvdJGFFLSKDCaY3BJev4Irzf/view?usp=sharing
file_id = '1oT9OJ2tRsvdJGFFLSKDCaY3BJev4Irzf'
url = f"https://drive.google.com/uc?id={file_id}"
output = 'vidcom_example.zip'
!gdown $url -O $output
!unzip $output

In [ ]:
import numpy as np
import seaborn as sns
import random
from scipy import stats
import matplotlib.pyplot as plt

from scandy.models.LocationModel import LocationModel
from scandy.models.ObjectModel import ObjectModel
from scandy.utils.dataclass import Dataset
import scandy.utils.functions as uf

from neurolib.utils.parameterSpace import ParameterSpace
from neurolib.optimize.evolution import Evolution

# Model optimization and quantitative comparison

In this example, we show how we can find suitable parameters for a model with evolutionary optimization, such that models can be compared quantitatively. 

## Dataset and model initialization
The parameter optimization should be done on the training set, but for this example we demonstrate how to do this on a single video.

In [ ]:
vidlist = sorted(['dance01', 'dance02', 'field03', 'foutain02', 'garden04', 'garden06', 'garden07', 'garden09', 'park01', 'park06', 'park09', 'road02', 'road04', 'road05', 'robarm01', 'room01', 'room02', 'room03', 'tommy02', 'uscdog01', 'walkway01', 'walkway02', 'walkway03'])
random.seed(12345)
# this split is used for the results in the manuscript
trainlist = sorted(random.sample(vidlist, 10))
testlist = sorted([vidname for vidname in vidlist if vidname not in trainlist])
print("trainlist = ", trainlist, "\ntestlist =", testlist)
# but here we only use one example video...

datadict = {
    "PATH": "VidCom_example/",  # previously downloaded & extracted dataset  
    'FPS' : 30,
    'PX_TO_DVA' : 0.06,
    'FRAMES_ALL_VIDS' : 300,
    'gt_foveation_df' : 'VidCom_GT_fov_df.csv',
    'trainset' : ["field03"]  # trainlist,
    # 'testset' : testlist,
}
VidCom = Dataset(datadict)

We only run it for a single model here, which is specified by the model family (MODEL) and the saliency information used (FEATURESET). 

In [ ]:
MODEL = "obj" # "loc" or "obj"
# we can vary the saliency map, "molin" corresponds to low-level features
FEATURESET = "molin" # "molin", "TASEDnet" (high-level), or "None" (center bias only)

## Implement the fitness function

With this information, we can specify the model we want to investigate. The parameters are then selected for each run based on the evolutionary optimization strategy. 

But before we specify the parameter space, we define an optimization function, which we will pass to the `Evolution` class from `neurolib` (powered by the `deap` and `pypet` libraries).

We specify a fitness function $\mathcal{F}$ to evaluate how well the simulated scanpaths match the human data. In this example we evaluate each parameter configuration by compating the summary statistics of the saccade amplitude and foveation duration of the simulated scanpaths with the human ground truth using the Kolmogorov-Smirnov test.

In [ ]:
# ground truth summary statistics from the training set, this is used to evaluate the fitness
gt_amp_dva = VidCom.train_foveation_df["sac_amp_dva"].dropna().values
gt_dur_ms = VidCom.train_foveation_df["duration_ms"].dropna().values

fig, axs = plt.subplots(1,2,dpi=100, figsize=(8,3), sharey=True)
sns.histplot(data=np.log10(gt_dur_ms), kde=False, ax=axs[0], bins=40)
axs[0].set_xticks([1,2,3,4]); axs[0].set_xticklabels([10,100,1000,10000])
axs[0].set_xlabel('Foveation duration [ms]'); axs[0].set_ylabel('Count')
sns.histplot(data=gt_amp_dva, kde=False, ax=axs[1], bins=60, label="training set")
axs[1].set_xlabel('Saccade amplitude [dva]')
sns.despine(); plt.legend()
plt.tight_layout(); plt.show()

This is implemented in the following optimization function, where the model is initialized in every run with the parameters selected by the evolutionary optimization strategy and then run for all videos in the training set (in our example just a single video).
The resulting scanpath(s) are then compared to the human data according to the fitness function $\mathcal{F}$ and the results are stored in the HDF file.

In [ ]:
def optimize_me(traj):
    ind = evolution.getIndividualFromTraj(traj)
    # create model of the given model family
    if MODEL == "obj":
        model = ObjectModel(VidCom)
    elif MODEL == "loc":
        model = LocationModel(VidCom)
    else:
        raise Exception(f"The only implemented models here are obj and loc!")
    model.params["centerbias"] = "anisotropic_default"
    model.params["featuretype"] = FEATURESET
    # free model parameters, varied in evolution
    model.params["ddm_thres"] = ind.ddm_thres
    model.params["ddm_sig"] = ind.ddm_sig
    model.params["att_dva"] = ind.att_dva
    model.params["ior_decay"] = ind.ior_decay
    # IOR parameters depend on the model...
    if MODEL == "obj":
        model.params["ior_inobj"] = ind.ior_inobj
    else:
        model.params["ior_dva"] = ind.ior_dva

    # usually run model on all videos of the training set for multiple random seeds
    # model.run("train", seeds=[s for s in range(1, 13)])
    # ...but for demonstration purposes, we only run the model on one video once:
    model.run("field03", seeds=[s for s in [1]])

    model.evaluate_all_to_df()  # creates model.result_df
    sim_dur_ms = model.result_df["duration_ms"].dropna().values
    sim_amp_dva = model.result_df["sac_amp_dva"].dropna().values

    # evaluate fitness
    ks_amp, _ = stats.ks_2samp(gt_amp_dva, sim_amp_dva)
    ks_dur, _ = stats.ks_2samp(gt_dur_ms, sim_dur_ms)
    fitness_tuple = (ks_dur, ks_amp)

    # we can store more information in the HDF file by returning a dictionary
    res_dict = model.get_fovcat_ratio()
    
    return fitness_tuple, res_dict

## Specify parameters and run optimization

We can now specify the parameter space for the optimization. We use the `ParameterSpace` class from `neurolib` to give a range for all free model parameters.

In [ ]:
obj_pars = ParameterSpace(
    ["ddm_thres", "ddm_sig", "att_dva", "ior_decay", "ior_inobj"],
    [[1.0, 3.0], [0.05, 0.25], [5, 20], [30, 300], [0.4, 1.0]],
)

Now we have everything set up to actually run the evolutionary optimization. As specified in `optimize_me`, we only run it for one video and one random seed. 

The processes are parallelized as much as your system allows for it. 

In [ ]:
evolution = Evolution(
    optimize_me,
    obj_pars,
    weightList=[-1.0, -1.0], #weights foveation duration and saccade amplitude equally
    filename="ex2_obj_sglvid.hdf",
    POP_INIT_SIZE=6,  # better: 64
    POP_SIZE=6,  # better: 32
    NGEN=5,  # better: 50
)
# verbose means it creates multiple plots for each generation
evolution.run(verbose = True)

Usually, this evolution would be run on all videos of the trainingset, with multiple seeds, higher population sizes and for more generations. Hence, this typically runs best on a high-performance computing cluster.

The results can be saved in a DILL file with the following command:

In [ ]:
evolution.saveEvolution("saved_evolution_ex2_obj_sglvid.dill")

## Further analysis / functional evaluation

To actually evaluate how well the simulated scanpaths correspond to human exploration behavior, we recommend to look into the functional scanpath comparison. The basis of this are the ratios of time that are spent in each foveation category, which we saved in this example in the result dictionary. 

The outputs from the evolution in the hdf file can be accessed in in a convenient way as pandas DataFrame:

In [ ]:
df_evol = evolution.dfEvolution(outputs=True)
# show the tree best parameter combinations with their fitness score and the stored results 
df_evol.sort_values(by=["score"], ascending=False).iloc[0:3]

For a more in depth analysis of the foveation categories, please refer to the `manuscript_results.ipynb` notebook, where the full analysis is shown in detail.

_________________________________________